In [34]:
from extract_parquet import read_parquet,extract_file_path
import cmbagent 
from make_prompts import make_prompt, extract_python_definition
from ins import evaluate
import os

In [2]:
# Example with a single file
df = read_parquet("data/public-00000-of-00001.parquet")


In [41]:
# --- usage ---------------------------------------------------------------
i = 1                                        # whichever row you want
prompt = make_prompt(df.iloc[i])             # combined string for that row
print(prompt)

### Problem
In cosmology, large-scale cosmological dark-matter halo fields are biased tracers of the underlying Gaussian matter density $\delta_m$. Assume we have a sample $\delta_m$. We simulate a halo number density field by taking $n(\mathbf{x}) = \bar{n}\max(0,1+b\delta_m(\mathbf{x}))$, where bare number density $\bar{n}$ and bare bias $b$ are specified constants. What is the bias of the sampled halo field? Derive an equation to evaluate the bias which depends on the bare bias and the variance in each pixel.

### Requirements
Provide the answer in the form of the \texttt{python} code. Implement the following function.
\begin{python}
#let b_in stand for bare bias
def b_eff(sigma: float, b_in:float) -> float:
    pass
\end{python}


In [43]:
reference_code = extract_python_definition(df.reference_implementation.iloc[i])

In [45]:
results = cmbagent.one_shot(prompt,
                   max_rounds=50,
                   agent='engineer',
                   engineer_model='gpt-4.1',
                   researcher_model='gpt-4.1',
                   #engineer_model='gemini-2.5-pro-exp-03-25',
                            
                   work_dir="./evals",
                  )

/Users/antoidicherianlonappan/Software/cmbagent_env/lib/python3.12/site-packages/autogen/agentchat/conversable_agent.py:129: UserWarning: Update function string contains no variables. This is probably unintended.
  warnings.warn("Update function string contains no variables. This is probably unintended.")


_User (to chat_manager):

### Problem
In cosmology, large-scale cosmological dark-matter halo fields are biased tracers of the underlying Gaussian matter density $\delta_m$. Assume we have a sample $\delta_m$. We simulate a halo number density field by taking $n(\mathbf{x}) = \bar{n}\max(0,1+b\delta_m(\mathbf{x}))$, where bare number density $\bar{n}$ and bare bias $b$ are specified constants. What is the bias of the sampled halo field? Derive an equation to evaluate the bias which depends on the bare bias and the variance in each pixel.

### Requirements
Provide the answer in the form of the \texttt{python} code. Implement the following function.
\begin{python}
#let b_in stand for bare bias
def b_eff(sigma: float, b_in:float) -> float:
    pass
\end{python}

--------------------------------------------------------------------------------

Next speaker: engineer


>>>>>>>> USING AUTO REPLY...
             Model    agent    Cost  Prompt Tokens  Completion Tokens  Total Tokens
gpt-4.1-20

In [46]:
results.keys()

dict_keys(['chat_history', 'final_context', 'initialization_time', 'execution_time'])

In [47]:
results['execution_time']

49.165186166763306

In [48]:
filepath = os.path.join(results['final_context']['work_dir'],extract_file_path(results['final_context']['current_codebase']))

In [49]:
with open(filepath, "r", encoding="utf-8") as f:
    candidate_code = f.read()

In [50]:
evaluate(reference_code,candidate_code)

╭─ code_match_with_similarity (1 sample): openai/gpt-4.1-2025-04-14 ──────────────────────────────────────────────╮
│ reference: from scipy.stats import norm                                                      dataset: (samples) │
│ #let b_in stand for bare bias                                                                                   │
│ def b_eff(sigma: float, b_in:float) -> float:                                                                   │
│     alpha = sigma*abs(b_in)                                                                                     │
│     return b_in*norm.cdf(1/alpha)/(norm.cdf(1/alpha)+alpha*norm.pdf(1/alpha)), candidate: #                     │
│ filename: codebase/b_eff.py                                                                                     │
│ import numpy as np                                                                                              │
│ from scipy.stats import norm                                                                                    │
│                                                                                                                 │
│ def b_eff(sigma: float, b_in: float) -> float:                                                                  │
│     r"""                                                                                                        │
│     Compute the effective bias of a halo field generated by thresholding a Gaussian matter                      │
│ density field.                                                                                                  │
│                                                                                                                 │
│     The halo number density field is defined as:                                                                │
│         n(x) = nbar * max(0, 1 + b_in * delta_m(x))                                                             │
│     where delta_m(x) is a zero-mean Gaussian random field with variance sigma^2.                                │
│                                                                                                                 │
│     The effective bias is defined as:                                                                           │
│         b_eff = <n(x) * delta_m(x)> / (nbar * <delta_m(x)^2>)                                                   │
│     where <...> denotes expectation over the Gaussian field.                                                    │
│                                                                                                                 │
│     Parameters                                                                                                  │
│     ----------                                                                                                  │
│     sigma : float                                                                                               │
│         Standard deviation of the Gaussian matter density field delta_m(x).                                     │
│         Units: dimensionless (overdensity).                                                                     │
│     b_in : float                                                                                                │
│         Bare bias parameter used in the halo field definition.                                                  │
│         Units: dimensionless.                                                                                   │
│                                                                                                                 │
│     Returns                                                                                                     │
│     -------                                                                                                     │
│     b_eff : float                                     

{'same_io_dynamic': 0.0, 'content_similarity': 0.11207335710646969}